## Transfer Raw Scene Viewing Retrieval Files to USB from Host Computer (.ps1)

VERSION: `Resting_Eyetrack_Auto_Push`

### Advanced Explanation

The script sets up data transfer from `$sourceBase` to `$destinationBase` for directories updated in the last 10 hours. The `$blocks` array (1 to 3) is iterated over, constructing `$sourcePath` for each block. It fetches directories in `$sourcePath` modified after `$currentDate.AddHours(-10)` using `Get-ChildItem` and `Where-Object`.

If no directories are found for a block (`$rawdata.Count -eq 0`), the block number is added to `$missingBlocks`. For directories found, the script calculates their size in MB and appends this info to `$dirInfo`.

After collecting directory data, the script prompts the user for confirmation to proceed with the transfer. If there are missing blocks and the user still opts to continue, it starts the transfer process. The script then iterates again through each block, checking against `$missingBlocks`. It constructs source and destination paths, fetches the relevant directories again, and if they meet the time criteria, copies them to the destination using `Copy-Item`. The script outputs each copy operation and aborts if the user opts out at any confirmation stage.

### User-Friendly Explanation

This script is designed to transfer specific folders from one location on your computer to another. It looks for folders within certain subfolders ("Last Block # Retrieval Task Fixed_deploy\results") that have been updated in the last 10 hours.

First, it checks which of these subfolders have been recently updated. If any are not updated, it notes them as missing. Then, it asks if you want to move the updated folders to a new location on a USB drive. If you agree, but there are some folders missing, it asks again to confirm whether to proceed without those folders. If you still agree, it moves the updated folders to the new location, telling you what it's doing along the way. If you say 'no' at any point, it stops and doesn't move anything.

### Oversimplified Explantion

This script is adjusted for retrieval blocks. It is currently not in use because the computer for the post scan task uses powershell version 1 which is not compatible. (It's 2002 I wasn't even born yet)

In [ ]:
$sourceBase = "C:\Documents and Settings\UT\Desktop\Olsen Lab\Final Deployed Tasks Retrieval Aug 29"
$destinationBase = "E:\SceneViewing_DAY3"
$currentDate = Get-Date
$blocks = 1..3
$missingBlocks = @()
$dirInfo = @()

foreach ($block in $blocks) {
    $sourcePath = Join-Path -Path $sourceBase -ChildPath ("Last Block $block Retrieval Task Fixed_deploy\results")
    $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

    if ($rawdata.Count -eq 0) {
        $missingBlocks += $block
        continue
    } else {
        foreach ($dir in $rawdata) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
            $dirSizeMBStr = $dirSizeMB.ToString() + "M"
            $dirInfo += "[$($dir.Name): $($dirSizeMBStr)]"
        }
    }
}

$dirInfoStr = $dirInfo -join ", "
$userInput = Read-Host "Do you want to move the following directories to their respective folders in the USB? Directories: $dirInfoStr [Y/N]"

if ($userInput -eq "Y") {
    if ($missingBlocks.Count -gt 0) {
        $missingBlockStr = $missingBlocks -join ", "
        $continueAnyway = Read-Host "Unexpected event: block(s) [$missingBlockStr] missing. Continue anyway? [Y/N]"
        if ($continueAnyway -ne "Y") {
            Write-Output "Operation aborted due to missing blocks."
            exit
        }
    }

    foreach ($block in $blocks) {
        if ($missingBlocks -contains $block) {
            Write-Output "Skipping block $block as it is missing."
            continue
        }

        $sourcePath = Join-Path -Path $sourceBase -ChildPath ("Last Block $block Retrieval Task Fixed_deploy\results")
        $destinationPath = Join-Path -Path $destinationBase -ChildPath ("Retrieval_Block$block")
        $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

        foreach ($dir in $rawdata) {
            $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to $($destinationPath)"
        }
    }
} else {
    Write-Output "Operation cancelled by user."
}
